## I. Pre-treatment of compounds

In [1]:
# !pip install pubchempy
# !pip install chemspipy
# from chemspipy import ChemSpider
# import pubchempy

In [1]:
valid = 0
infer_valid = 0

In [2]:
import urllib
import pandas as pd 
import numpy as np
from tqdm import tqdm
from sklearn import preprocessing
import seaborn as sns
import pickle
import re 

### 1. Create a collection of all compounds      
      
#### 1.1 Getting the compounds needed for food      
- sr_food_nur5672/FoodDB crawl data.xlsx: foodDB crawl results http://foodb.ca/foods

In [3]:
web_food = pd.read_excel('../data/FoodDB爬取数据.xlsx')#('/home/mw/input/web_food3967/初始完整版.xlsx')
display(web_food.head())
web_food = web_food.drop('id', axis=1)
pattern = r"[a-z A-Z]"
cond_a = web_food['平均值'].astype(str).str.contains(pattern, regex=True)
cond_b = web_food['最小值'].astype(str).str.contains(pattern, regex=True)
cond_c = web_food['最大值'].astype(str).str.contains(pattern, regex=True)
cond = zip(cond_a.to_list(), cond_b.to_list(), cond_c.to_list())
cond = map(lambda x: x[0] or x[1] or x[2], cond)
cond = pd.Series(list(cond))
web_food = web_food[~cond]
# Find a list of unique compounds
comlist_web = web_food['化合物名称'].tolist()
comlist_web = set(comlist_web)

,id,食物名称,化合物名称,最小值,最大值,平均值,计量单位
0,53957f21209da71a9fa82c25784c8317,Alfalfa,Ash,0.4,3100,1166.8,mg/100g
1,53957f21209da71a9fa82c25784c8317,Alfalfa,Carbohydrate,200,41800,19500,mg/100g
2,53957f21209da71a9fa82c25784c8317,Alfalfa,Fat,700,12600,6650,mg/100g
3,53957f21209da71a9fa82c25784c8317,Alfalfa,Fatty acids,0,400,100,mg/100g
4,53957f21209da71a9fa82c25784c8317,Alfalfa,Fiber (dietary),0,7900,2450,mg/100g


In [4]:
# Formatting & converting names to lower case
food_compound = web_food[["食物名称","化合物名称", "平均值", "计量单位"]]
food_compound.columns = ["food_name", "compound_name", "standard_content", "orig_unit"]
food_compound = food_compound.reset_index()

food_compound['food_lower'] = food_compound['food_name'].apply(lambda x: x.lower())
food_compound['cmp_lower'] = food_compound['compound_name'].apply(lambda x: x.lower())

# Data scaling
scaler1=preprocessing.MinMaxScaler()
food_compound = food_compound.join(food_compound.groupby(['cmp_lower'])[['standard_content']].apply(lambda x: pd.DataFrame(scaler1.fit_transform(x), index=x.index, columns=['scaler'])))
food_compound['scaler'] = 0.01 + food_compound['scaler']*0.99

unique_food_compound = list(set(food_compound['cmp_lower']))
unique_food = list(set(food_compound['food_lower']))

display(food_compound.head())
print(f'食物{len(unique_food)}：共涉及化合物{len(unique_food_compound)}种')
# food_compound.to_csv('final_food_compound.csv', index = False)

,index,food_name,compound_name,standard_content,orig_unit,food_lower,cmp_lower,scaler
0,0,Alfalfa,Ash,1166.8,mg/100g,alfalfa,ash,0.025652
1,1,Alfalfa,Carbohydrate,19500,mg/100g,alfalfa,carbohydrate,0.202252
2,2,Alfalfa,Fat,6650,mg/100g,alfalfa,fat,0.075373
3,3,Alfalfa,Fatty acids,100,mg/100g,alfalfa,fatty acids,0.010825
4,4,Alfalfa,Fiber (dietary),2450,mg/100g,alfalfa,fiber (dietary),0.101210


食物707：共涉及化合物3608种


In [6]:
sns.regplot(
    x= food_compound['standard_content'].astype(float),
    y= food_compound['scaler'],
    fit_reg=False
    )

<Figure size 432x288 with 1 Axes>

#### 1.2 Access to Diseases-Compounds      
      
Diseases and compounds: http://ctdbase.org/downloads/#cd      
![Image Name](https://cdn.kesci.com/upload/image/riwf4p2by1.jpeg?imageView2/0/w/960/h/960)

In [5]:
disease_compound = pd.read_csv('../data/ctd_com_disease.csv')
disease_compound = disease_compound[~disease_compound['InferenceScore'].isna()]

# Disease and compound relationships based on inference score
disease_compound['cmp_lower'] = disease_compound['# ChemicalName'].apply(lambda x: str(x).lower())
disease_compound['disease_lower'] = disease_compound['DiseaseName'].apply(lambda x: str(x).lower())

## Log transform the inference score feature and scale it to between 0 and 1
# min_max_scaler = preprocessing.MinMaxScaler()
# map_values = np.log(np.array(disease_compound['InferenceScore'].values))
# inference_score = np.array(min_max_scaler.fit_transform(map_values.reshape([len(map_values),1]))).reshape([1,len(map_values)])

#disease_compound['scaler_infer'] = 0.01 + np.array(list(inference_score[0]))*0.99 
disease_compound['scaler_infer'] = disease_compound['InferenceScore']

In [8]:
sns.regplot(
    x= disease_compound['InferenceScore'],
    y= disease_compound['scaler_infer'],
    fit_reg=False,
    scatter_kws={'alpha':0.15}
    )

In [6]:
disease_compound = disease_compound[disease_compound['scaler_infer']>infer_valid]
unique_disease_compound_ctd = set(disease_compound['cmp_lower'])
print(f"疾病{len(set(disease_compound['disease_lower']))}：共涉及化合物{len(unique_disease_compound_ctd)}种")
display(disease_compound.head())
# disease_compound.to_csv('final_disease_compound.csv', index = False)

疾病5851：共涉及化合物14133种


,Unnamed: 0,# ChemicalName,CasRN,DiseaseName,DirectEvidence,InferenceScore,cmp_lower,disease_lower,scaler_infer
1,1,10074-G5,NaN,Adenocarcinoma,NaN,4.08,10074-g5,adenocarcinoma,4.08
2,2,10074-G5,NaN,Adenocarcinoma of Lung,NaN,4.30,10074-g5,adenocarcinoma of lung,4.30
3,3,10074-G5,NaN,Alopecia,NaN,4.50,10074-g5,alopecia,4.50
4,4,10074-G5,NaN,Androgen-Insensitivity Syndrome,NaN,6.86,10074-g5,androgen-insensitivity syndrome,6.86
5,5,10074-G5,NaN,Astrocytoma,NaN,4.93,10074-g5,astrocytoma,4.93


#### 1.4 Compound integration

In [7]:
all_compound = unique_food_compound.copy()
all_compound.extend(unique_disease_compound_ctd)

all_inchikey = pd.DataFrame({
    'compound': all_compound,
    'inchikey': np.nan,
    'source': np.nan
}).drop_duplicates()
all_inchikey['lower'] = all_inchikey['compound'].apply(lambda x: x.lower())
all_casrn = all_inchikey.copy()
all_casrn.columns=["compound", "casrn", "source", "lower"]
print(f'共有{len(all_inchikey)}种化合物需要匹配')
display(all_inchikey.head())
display(all_casrn.head())

共有17273种化合物需要匹配


,compound,inchikey,source,lower
0,zeanoside c,NaN,NaN,zeanoside c
1,gamma-butyrolactone,NaN,NaN,gamma-butyrolactone
2,(s)-(+)-ethyl-2-methylbutanoate,NaN,NaN,(s)-(+)-ethyl-2-methylbutanoate
3,"dg(16:0/20:2(11z,14z)/0:0)",NaN,NaN,"dg(16:0/20:2(11z,14z)/0:0)"
4,"tg(15:0/16:0/20:3(8z,11z,14z))[iso6]",NaN,NaN,"tg(15:0/16:0/20:3(8z,11z,14z))[iso6]"


,compound,casrn,source,lower
0,zeanoside c,NaN,NaN,zeanoside c
1,gamma-butyrolactone,NaN,NaN,gamma-butyrolactone
2,(s)-(+)-ethyl-2-methylbutanoate,NaN,NaN,(s)-(+)-ethyl-2-methylbutanoate
3,"dg(16:0/20:2(11z,14z)/0:0)",NaN,NaN,"dg(16:0/20:2(11z,14z)/0:0)"
4,"tg(15:0/16:0/20:3(8z,11z,14z))[iso6]",NaN,NaN,"tg(15:0/16:0/20:3(8z,11z,14z))[iso6]"


### 2. Get inchikey with casrn      
      
All compound sources that need to be matched: food compound & disease compound(ctd)      
      
Get sources:      
1. after processing garbled compound, download compound.csv from foodDB to match inchikey with casrn      
2. download data from CTD to match casrn      
3. call pubchem's official API to query inchikey      
3. history retrieved food_inchikey (foodCompDisease4478/compounds_pubchem.csv) + disease_inchikey (foodCompDisease4478/pubchem_com.csv)      
4. call chemspider API query      
5. manually supplemented to achieve a final match rate of around 90%      


In [8]:
def merge_info(all_cmp, inchikey, source, target='inchikey'):
    inchikey.columns = ['lower', source]
    result = all_cmp.merge(inchikey, on='lower', how='left')
    to_update = (~result[source].isnull())&(result[target].isnull())
    to_add = (~result[source].isnull())&(~result[target].isnull())
    # Direct updates without target values
    result.loc[to_update, target] = result.loc[to_update, source]
    result.loc[to_update, 'source'] = source
    # Append backwards if the target value already exists
    add_df = result[to_add].reset_index(drop=True)
    add_df[target] = add_df[source]
    add_df['source'] = source
    # Data merging
    result = pd.concat([result, add_df], axis=0)
    result = result.drop_duplicates()
    complete = len(set(result[result[target].isnull()==False]['compound']))
    total = len(set(all_cmp['compound']))
    print(f"已匹配{target}{complete}个（共{result.shape[0]}行），剩余{total-complete}个, 完整度：{round(complete/total*100, 2)}%")
    incomplete_df = result[result[target].isnull()]['lower']
    return result[['compound', 'lower', target, 'source']], incomplete_df

#### 2.1 foodDB Official Website Download Data Match      
      
FooDB CSV file from https://foodb.ca/downloads

In [10]:
food_unmatch = all_inchikey[['compound']].drop_duplicates()
# Resolve garbled code issues
food_unmatch['connect_name'] = food_unmatch['compound'].apply(lambda x: x.replace("&#39;", "'"))
food_unmatch['connect_name'] = food_unmatch['connect_name'].apply(lambda x: x.replace("&gt;", ">"))
food_unmatch['connect_name'] = food_unmatch['connect_name'].apply(lambda x: x.replace("&quot;", '"'))

#Rewrite the garbled parts and associate them in lower case
db_cmp = pd.read_csv('/home/mw/input/foodCompDisease4478/db_Compound.csv')
db_cmp['lower'] = db_cmp['name'].apply(lambda x: x.lower())
food_unmatch['lower'] = food_unmatch['connect_name'].apply(lambda x: x.lower()) 
fdb = food_unmatch.merge(db_cmp[['lower', 'moldb_smiles', 'description']], on='lower', how='inner')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3,4,5,6,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
# When associating with full volume data, garbled lower case names are still used
inchikey_after_fdb, incomplete_after_fdb = merge_info(all_inchikey, fdb[~fdb['moldb_smiles'].isnull()][['compound', 'moldb_smiles']], 'fdb')
casrn_after_fdb, incomplete_after_fdb_casrn = merge_info(all_casrn, fdb[~fdb['description'].isnull()][['compound', 'description']], 'fdb', 'casrn')

已匹配inchikey4349个（共17273行），剩余12924个, 完整度：25.18%
已匹配casrn2491个（共17273行），剩余14782个, 完整度：14.42%


#### 2.2 from CTD Associated Diseases casrn

In [12]:
ctd_disease = disease_compound[~disease_compound['CasRN'].isnull()][['cmp_lower', 'CasRN']].reset_index(drop=True)
casrn_after_ctd, incomplete_after_ctd_casrn = merge_info(casrn_after_fdb, ctd_disease, 'ctd', 'casrn')

已匹配casrn8793个（共18253行），剩余8480个, 完整度：50.91%


#### 2.3 From the pubchem API

In [13]:
def get_pubchem_info(compound_list):
    def get_casrn(x):
        matched = re.match('(\d{2,7}-\d\d-\d)', x)
        result = matched.group(1) if matched else '--'
        return(result)
    compound_name = []
    cid = []
    inchikey = []
    casrn = []
    fail = 0
    for idx, i in tqdm(enumerate(compound_list)):
        target = pubchempy.get_compounds(i, 'name')
        if target==[]:
            fail+=1
        else:
            compound_name.append(i)
            target_cid = target[0]
            cid.append(target_cid.cid)
            inchikey.append(target_cid.inchikey)
            cas_list = list(set(map(get_casrn, target_cid.synonyms)) - set(['--']))
            if len(cas_list)>0:
                casrn.append(cas_list[0])
            else:
                casrn.append(np.nan)
        if (idx//50==idx/50) or (idx==len(compound_list)-1):
            result = pd.DataFrame({"cid": cid, "lower":compound_name, "inchikey": inchikey, "casrn": casrn})
            result.to_csv('success_inchikey_temp.csv')

    print(f'累计查找失败{fail}次')
    return result, fail

In [14]:
# incomplete = list(incomplete_after_ctd_casrn)
# incomplete.extend(incomplete_after_fdb)
# incomplete = list(set(incomplete))
# pickle.dump(incomplete, open('incomplete.pkl', 'wb'))
# # The call record is saved in success_inchikey_casrn.csv, type new. API calls can be made by uncommenting the following code (which takes longer)
#pubchemAPI, pubchemAPI_null = get_pubchem_info(incomplete)

In [16]:
for_casrn = pd.read_csv('../data/success_inchikey_casrn.csv')
for_gen = pd.read_csv('../data/success_inchikey_gen.csv').drop(columns=['Unnamed: 0'])
pubchemAPI = pd.concat([for_casrn, for_gen], axis=0)
pubchemAPI_casrn = pubchemAPI[~pubchemAPI['casrn'].isnull()]
inchikey_after_pubchem, incomplete_after_pubchem = merge_info(inchikey_after_fdb, pubchemAPI[['lower', 'inchikey']], 'pubchemAPI')
casrn_after_pubchem, incomplete_after_pubchem_casrn = merge_info(casrn_after_ctd, pubchemAPI_casrn[['lower', 'casrn']], 'pubchemAPI', 'casrn')

已匹配inchikey14089个（共19656行），剩余3184个, 完整度：81.57%
已匹配casrn12333个（共26546行），剩余4940个, 完整度：71.4%


#### 2.4 Matching from historical results      
      
- Associated with inchikey

In [17]:
pre_food_inchikey = pd.read_csv('../data/compounds_pubchem.csv')
pre_food_inchikey['lower'] = pre_food_inchikey['name'].apply(lambda x: x.lower())

pre_disease_inchikey = pd.read_csv('../data/pubchem_com.csv')
pre_disease_inchikey = pre_disease_inchikey[pre_disease_inchikey['cmpdname'].isnull()==False]
pre_disease_inchikey['lower'] = pre_disease_inchikey['cmpdname'].apply(lambda x: x.lower())

history = pd.concat([pre_food_inchikey[['lower', 'inchikey']], pre_disease_inchikey[['lower', 'inchikey']]], axis = 0)
history.columns = ['lower', 'history']
history_ready = history[history['lower'].isin(incomplete_after_pubchem)]
inchikey_after_history, incomplete = merge_info(inchikey_after_pubchem, history_ready, 'history')

已匹配inchikey14097个（共19656行），剩余3176个, 完整度：81.61%


#### 2.5 Calling the official chemspider API

In [19]:
# 调用记录保存在chemspider.csv中，API调用可取消下列注释代码（用时较久）
# my_key = '9ADUvpFHGPrWmp46bOxNvracW2Vz1Pml'
# cs = ChemSpider(my_key)

# spider = []
# spider_lower = []
# for cmp in incomplete_after_fdb:
#     spider_lower.append(cmp)
#     searched = cs.search(cmp)
#     if len(searched)>0:
#         spider.append(searched[0].inchikey)
#     else:
#         spider.append(np.nan)

# chemspider = pd.DataFrame({"lower": spider_lower, "spider": spider})
# chemspider = chemspider[chemspider['spider'].isnull()==False]
# chemspider.to_csv('/home/mw/project/inchikey中间表/chemspider.csv', index = False)

In [21]:
chemspider = pd.read_csv('../data/chemspider.csv')
inchikey_after_spider, incomplete_after_spider = merge_info(inchikey_after_history, chemspider[['lower', 'spider']], 'spider')

已匹配inchikey14176个（共19659行），剩余3097个, 完整度：82.07%


#### 2.3 Manual check matching

In [23]:
manul = pd.read_csv('../data/manul_inchikey.csv', usecols=['compound_name', 'inchikey', 'info', 'casrn'])
manul_exact = manul[(manul['info'].isnull())] # 删除近似匹配的部分
inchikey_after_manul, incomplete_after_manul = merge_info(inchikey_after_spider, manul_exact[['compound_name', 'inchikey']][(~manul_exact['inchikey'].isnull())], 'manul')
casrn_after_manul, incomplete_after_manul_casrn = merge_info(casrn_after_pubchem, manul_exact[['compound_name', 'casrn']][(~manul_exact['casrn'].isnull())], 'manul', 'casrn')

已匹配inchikey14704个（共19674行），剩余2569个, 完整度：85.13%
已匹配casrn12532个（共25841行），剩余4741个, 完整度：72.55%


#### Integration of results

In [24]:
whole_inchikey = inchikey_after_manul[inchikey_after_manul['inchikey'].isnull()==False]
whole_inchikey = whole_inchikey.groupby(['compound','lower', 'inchikey'])['source'].apply(", ".join).reset_index()

whole_casrn = casrn_after_manul[casrn_after_manul['casrn'].isnull()==False]
whole_casrn = whole_casrn.groupby(['compound','lower', 'casrn'])['source'].apply(", ".join).reset_index()

In [25]:
count_inchi_match_cmp = whole_inchikey.groupby('lower').count()
count_inchi_match_cmp[count_inchi_match_cmp['compound']>1]
whole_inchikey[whole_inchikey['lower']=='1,2-disinapoylgentiobiose']

,compound,lower,inchikey,source
297,"1,2-disinapoylgentiobiose","1,2-disinapoylgentiobiose",MBGNTECDWBKCKH-HXDPBKTRSA-N,pubchemAPI
298,"1,2-disinapoylgentiobiose","1,2-disinapoylgentiobiose",MBGNTECDWBKCKH-KQQUZDAGSA-N,fdb


In [26]:
# inchikey_after_manul.to_csv('/home/mw/project/inchikey中间表/inchikey_info_withNA.csv', index=False)
# whole_inchikey.to_csv('/home/mw/project/inchikey中间表/inchikey_info.csv', index = False)
# whole_casrn.to_csv('/home/mw/project/inchikey中间表/casrn_info.csv', index = False)

# whole_inchikey = pd.read_csv('/home/mw/project/inchikey中间表/inchikey_info.csv'
# whole_casrn = pd.read_csv('/home/mw/project/inchikey中间表/casrn_info.csv'


### 3. match inchikey      
      
#### 3.1 food_inchikey

In [27]:
food_compound_inchikey_new = pd.merge(food_compound, whole_inchikey, left_on='cmp_lower', right_on='lower', how='left')
food_compound_inchikey_new = food_compound_inchikey_new[['food_lower','cmp_lower', 'scaler', 'inchikey','source']].drop_duplicates()
food_compound_inchikey_new = food_compound_inchikey_new.merge(whole_casrn[['lower', 'casrn']], left_on='cmp_lower', right_on='lower', how='left')
food_compound_inchikey_new = food_compound_inchikey_new[~((food_compound_inchikey_new['inchikey'].isnull()) & (food_compound_inchikey_new['casrn'].isnull()))]
print(f'Food Inchikey New {food_compound_inchikey_new.shape}')
print('基于 whole inchikey', len(list(set(food_compound_inchikey_new['cmp_lower']))), len(list(set(food_compound_inchikey_new['food_lower']))))
display(food_compound_inchikey_new.head())

Food Inchikey New (42906, 7)
基于 whole inchikey 3604 684


,food_lower,cmp_lower,scaler,inchikey,source,lower,casrn
2,alfalfa,fat,0.075373,IPCSVZSSVZVIGE-UHFFFAOYSA-N,pubchemAPI,fat,57-10-3
3,alfalfa,fatty acids,0.010825,FERIUCNNQQJTOY-UHFFFAOYSA-N,pubchemAPI,fatty acids,67254-79-9
6,alfalfa,water,0.342161,XLYOFNOQVPJJNP-UHFFFAOYSA-N,pubchemAPI,water,7732-18-5
7,alfalfa,nitrogen,0.623799,IJGRMHOSHXDMSA-UHFFFAOYSA-N,fdb,nitrogen,7727-37-9
8,alfalfa,l-leucine,0.019383,ROHFNLRQFUQHCH-UHFFFAOYSA-N,fdb,l-leucine,61-90-5


#### 3.2 disease_inchikey

In [28]:
# Link to inchikey information by compound name
disease_compound_inchikey_new = pd.merge(disease_compound, whole_inchikey,left_on='cmp_lower',right_on='lower',how='inner')
disease_compound_inchikey_new = disease_compound_inchikey_new[['cmp_lower', 'disease_lower', 'scaler_infer', 'inchikey']].drop_duplicates()
disease_compound_inchikey_new = disease_compound_inchikey_new.merge(whole_casrn[['lower', 'casrn']], left_on='cmp_lower', right_on='lower', how='left')

print(f'Disease Compound Inchikey New {disease_compound_inchikey_new.shape}')
print('compound:', len(set(disease_compound_inchikey_new['cmp_lower'])))
print('disease:', len(set(disease_compound_inchikey_new['disease_lower'])))
display(disease_compound_inchikey_new.head())
# disease_compound_inchikey_new.to_csv('/home/mw/project/inchikey中间表/disease_inchikey.csv', index = False)

Disease Compound Inchikey New (3381917, 6)
compound: 11568
disease: 5851


,cmp_lower,disease_lower,scaler_infer,inchikey,lower,casrn
0,10074-g5,adenocarcinoma,4.08,KMJPYSQOCBYMCF-UHFFFAOYSA-N,10074-g5,413611-93-5
1,10074-g5,adenocarcinoma of lung,4.30,KMJPYSQOCBYMCF-UHFFFAOYSA-N,10074-g5,413611-93-5
2,10074-g5,alopecia,4.50,KMJPYSQOCBYMCF-UHFFFAOYSA-N,10074-g5,413611-93-5
3,10074-g5,androgen-insensitivity syndrome,6.86,KMJPYSQOCBYMCF-UHFFFAOYSA-N,10074-g5,413611-93-5
4,10074-g5,astrocytoma,4.93,KMJPYSQOCBYMCF-UHFFFAOYSA-N,10074-g5,413611-93-5


## II. Building a matrix of food, disease relationships      
      
![Image Name](https://cdn.kesci.com/upload/image/rjmhfvxydw.jpeg?imageView2/0/w/960/h/960)

In [29]:
# food_compound_inchikey_new = pd.read_csv('/home/mw/project/inchikey中间表/food_inchikey.csv')
# disease_compound_inchikey_new = pd.read_csv('/home/mw/project/inchikey中间表/disease_inchikey.csv')
print('食物：', len(set(food_compound_inchikey_new['food_lower'])), '食物化合物：', len(set(food_compound_inchikey_new['cmp_lower'])))
print('疾病：', len(set(disease_compound_inchikey_new['disease_lower'])), '疾病化合物：', len(set(disease_compound_inchikey_new['cmp_lower'])))

with_score = disease_compound_inchikey_new[disease_compound_inchikey_new['scaler_infer'].isnull()==False]
print('inferenceScore 非空疾病：', len(set(with_score['disease_lower'])), '疾病化合物：', len(set(with_score['cmp_lower'])))

食物： 684 食物化合物： 3604
疾病： 5851 疾病化合物： 11568
inferenceScore 非空疾病： 5851 疾病化合物： 11568


In [30]:
# Compound-based association table
## Food compounds taken on the left, disease compounds on the right
left = food_compound_inchikey_new[['cmp_lower','inchikey', 'casrn']].drop_duplicates()
right = disease_compound_inchikey_new[['cmp_lower','inchikey', 'casrn']]

In [31]:
disease_food_linkbycomp = pd.merge(left, right[['cmp_lower', 'inchikey']].drop_duplicates(), on='inchikey',how='left')
disease_food_linkbycomp.loc[disease_food_linkbycomp['casrn'].isnull(), 'casrn']=532
disease_food_linkbycomp = pd.merge(disease_food_linkbycomp, right[['cmp_lower','casrn']].drop_duplicates(), on='casrn',how='left')

# Merge the two associated cmps
disease_food_linkbycomp['disease_cmp'] = disease_food_linkbycomp['cmp_lower_y'] 
disease_food_linkbycomp.loc[disease_food_linkbycomp['disease_cmp'].isnull(), 'disease_cmp'] = disease_food_linkbycomp['cmp_lower'] 
disease_food_linkbycomp = disease_food_linkbycomp[~disease_food_linkbycomp['disease_cmp'].isnull()][['cmp_lower_x', 'inchikey', 'casrn','disease_cmp']]

In [44]:
disease_compound_inchikey_new.head()

,cmp_lower,disease_lower,scaler_infer,inchikey,lower,casrn
0,10074-g5,adenocarcinoma,4.08,KMJPYSQOCBYMCF-UHFFFAOYSA-N,10074-g5,413611-93-5
1,10074-g5,adenocarcinoma of lung,4.30,KMJPYSQOCBYMCF-UHFFFAOYSA-N,10074-g5,413611-93-5
2,10074-g5,alopecia,4.50,KMJPYSQOCBYMCF-UHFFFAOYSA-N,10074-g5,413611-93-5
3,10074-g5,androgen-insensitivity syndrome,6.86,KMJPYSQOCBYMCF-UHFFFAOYSA-N,10074-g5,413611-93-5
4,10074-g5,astrocytoma,4.93,KMJPYSQOCBYMCF-UHFFFAOYSA-N,10074-g5,413611-93-5


In [47]:
# View data dimensions
food_inner = food_compound_inchikey_new[food_compound_inchikey_new['cmp_lower'].isin(disease_food_linkbycomp['cmp_lower_x'])]
disease_inner = disease_compound_inchikey_new[disease_compound_inchikey_new['cmp_lower'].isin(disease_food_linkbycomp['disease_cmp'])]
unique_food = set(food_inner['food_lower'])
unique_disease = set(disease_inner['disease_lower'])
print('食物：', len(unique_food), '疾病：', len(unique_disease))
print('化合物种类（食物命名）：', len(set(disease_food_linkbycomp['cmp_lower_x'])), '化合物种类（疾病命名）：', len(set(disease_food_linkbycomp['disease_cmp'])))
display(disease_food_linkbycomp.head())

食物： 684 疾病： 5840
化合物种类（食物命名）： 848 化合物种类（疾病命名）： 827


,cmp_lower_x,inchikey,casrn,disease_cmp
0,fat,IPCSVZSSVZVIGE-UHFFFAOYSA-N,57-10-3,palmitic acid
1,fatty acids,FERIUCNNQQJTOY-UHFFFAOYSA-N,67254-79-9,butyric acid
2,fatty acids,FERIUCNNQQJTOY-UHFFFAOYSA-N,67254-79-9,fatty acids
3,water,XLYOFNOQVPJJNP-UHFFFAOYSA-N,7732-18-5,water
4,nitrogen,IJGRMHOSHXDMSA-UHFFFAOYSA-N,7727-37-9,nitrogen


      
### 1. second classification

In [48]:
# View data dimensions
food_inner = food_compound_inchikey_new[food_compound_inchikey_new['cmp_lower'].isin(disease_food_linkbycomp['cmp_lower_x'])]
disease_inner = disease_compound_inchikey_new[disease_compound_inchikey_new['cmp_lower'].isin(disease_food_linkbycomp['disease_cmp'])]
unique_food = set(food_inner['food_lower'])
unique_disease = set(disease_inner['disease_lower'])
print('食物：', len(unique_food), '疾病：', len(unique_disease))
print('化合物种类（食物命名）：', len(set(disease_food_linkbycomp['cmp_lower_x'])), '化合物种类（疾病命名）：', len(set(disease_food_linkbycomp['disease_cmp'])))
display(disease_food_linkbycomp.head())

食物： 684 疾病： 5840
化合物种类（食物命名）： 848 化合物种类（疾病命名）： 827


,cmp_lower_x,inchikey,casrn,disease_cmp
0,fat,IPCSVZSSVZVIGE-UHFFFAOYSA-N,57-10-3,palmitic acid
1,fatty acids,FERIUCNNQQJTOY-UHFFFAOYSA-N,67254-79-9,butyric acid
2,fatty acids,FERIUCNNQQJTOY-UHFFFAOYSA-N,67254-79-9,fatty acids
3,water,XLYOFNOQVPJJNP-UHFFFAOYSA-N,7732-18-5,water
4,nitrogen,IJGRMHOSHXDMSA-UHFFFAOYSA-N,7727-37-9,nitrogen


In [51]:
# Build relationship matrices, view sparsity
disease_food_incom = pd.DataFrame(
    np.zeros((len(unique_food), len(unique_disease))),
    index = unique_food,
    columns = unique_disease)
    
for food in tqdm(disease_food_incom.index):
    food_cmp = food_inner[(food_inner['food_lower']==food) & (food_inner['scaler']>valid)]
    disease_cmp = list(set(food_cmp.merge(disease_inner, on='inchikey')['disease_lower']))
    disease_food_incom.loc[food,disease_cmp] = 1

sizes = disease_food_incom.shape[0]*disease_food_incom.shape[1]
rea_num = np.sum(np.sum(disease_food_incom))
print(f'valid={valid}：存在的匹配关系：{rea_num}, 稀疏度={round((1-rea_num/sizes)*100, 2)}%')

# Check the fetch
sparse_check = disease_food_incom.copy()
sparse_check.loc["sum"] =sparse_check.apply(lambda x:x.sum()) 
df_summary = sparse_check.T[1:]
df_summary['rate'] = (df_summary['sum']/len(unique_food)).apply(lambda x: round(x, 2))
sns.distplot(a=df_summary['rate'], kde=True)

100%|██████████| 684/684 [01:04<00:00, 10.68it/s]


valid=0：存在的匹配关系：3207298.0, 稀疏度=19.71%


<Figure size 432x288 with 1 Axes>

In [32]:
# Check the fetch
sparse_check_T = disease_food_incom.copy().T
sparse_check_T.loc["sum"] =sparse_check_T.apply(lambda x:x.sum()) 
df_summary_T = sparse_check_T.T[1:]
df_summary_T['rate'] = (df_summary_T['sum']/len(unique_food)).apply(lambda x: round(x, 2))
sns.distplot(a=df_summary_T['rate'], kde=True)

<Figure size 432x288 with 1 Axes>

In [33]:
cc = sparse_check_T.T
print(len(cc[cc['sum']==0]), len(cc))

1 684


In [34]:
# disease_food_incom.to_csv('/home/mw/project/特征中间表/original_food_disease.csv')

### 2. quantification

In [35]:
disease_food_infer = pd.DataFrame(
    np.zeros((len(unique_food), len(unique_disease))),
    index = unique_food,
    columns = unique_disease)

In [36]:
print(np.sum(np.sum(disease_food_infer==0)),disease_food_infer.shape, 684*5840)

3994560 (684, 5840) 3994560


In [37]:
for food in tqdm(disease_food_infer.index):
    food_cmp = food_inner[food_inner['food_lower']==food]
    display(food_cmp.head())
    # disease_cmp = list(set(food_cmp.merge(disease_inner, on='inchikey')['disease_lower']))
    # disease_food_infer.loc[food,disease_cmp] = 1
    break

  0%|          | 0/684 [00:00<?, ?it/s]

,food_lower,cmp_lower,scaler,inchikey,source,lower,casrn
7488,date,fat,0.014457,IPCSVZSSVZVIGE-UHFFFAOYSA-N,pubchemAPI,fat,57-10-3
7489,date,fatty acids,0.011155,FERIUCNNQQJTOY-UHFFFAOYSA-N,pubchemAPI,fatty acids,67254-79-9
7492,date,sugars,0.751173,CZMRCDWAGMRECN-UGDNZRGBSA-N,fdb,sugars,57-50-1
7493,date,starch,0.290137,WQZGKKKJIJFFOK-UHFFFAOYSA-N,fdb,starch,9005-25-8
7494,date,sucrose,0.413104,CZMRCDWAGMRECN-UHFFFAOYSA-N,fdb,sucrose,57-50-1


  0%|          | 0/684 [00:00<?, ?it/s]


In [39]:
disease_food_linkbycomp = pd.merge(food_inner[['food_lower', 'cmp_lower','scaler']], disease_food_linkbycomp,left_on='cmp_lower',right_on='cmp_lower_x',how='inner')

In [40]:
disease_food_linkbyinfer = pd.merge(
    disease_food_linkbycomp[['food_lower', 'cmp_lower_x', 'scaler','disease_cmp']].drop_duplicates(), 
    with_score[['cmp_lower', 'disease_lower', 'scaler_infer']].drop_duplicates(), left_on='disease_cmp', right_on='cmp_lower',how='inner')
unique_food = set(disease_food_linkbyinfer['food_lower'])
unique_disease = set(disease_food_linkbyinfer['disease_lower'])
print(disease_food_linkbyinfer.shape)
print('食物：', len(unique_food), '疾病：', len(unique_disease))
print('化合物种类（食物命名）：', len(set(disease_food_linkbyinfer['cmp_lower_x'])), '化合物种类（疾病命名）：', len(set(disease_food_linkbyinfer['disease_cmp'])))

(26115705, 7)
食物： 684 疾病： 5840
化合物种类（食物命名）： 848 化合物种类（疾病命名）： 827


In [41]:
disease_food_linkbyinfer.shape

(26115705, 7)

In [42]:
# Calculate relationship matrix scores
disease_food_linkbyinfer['values'] = disease_food_linkbyinfer['scaler']*disease_food_linkbyinfer['scaler_infer']
disease_food_infermatrix = disease_food_linkbyinfer.groupby(['food_lower', 'disease_lower'])['values'].agg('sum').reset_index()
disease_food_infer = disease_food_infermatrix.pivot(index = 'food_lower', columns = 'disease_lower', values = 'values')
nullV = np.sum(np.sum(disease_food_infer.isnull()))
disease_food_infer[disease_food_infer.isnull()]=0
temp_relation_infer = disease_food_linkbyinfer[['food_lower', 'disease_lower']].drop_duplicates()

sizes = disease_food_infer.shape[0]*disease_food_infer.shape[1]
print(f'存在的匹配关系：{temp_relation_infer.shape[0]}, 稀疏度={round((nullV/sizes)*100, 2)}%')

存在的匹配关系：3236956, 稀疏度=18.97%


In [43]:
print("矩阵1:0的比例",np.sum(np.sum((disease_food_infer != 0).astype(int)))/np.sum(np.sum((disease_food_infer == 0).astype(int))))

矩阵1:0的比例 4.272622636628107


In [44]:
# com for scaling, inference not processed
import matplotlib.pyplot as plt
plt.boxplot(np.array(disease_food_infer).reshape(-1,))
np.median(list(np.array(disease_food_infer).reshape(-1,)))
np.percentile(list(np.array(disease_food_infer).reshape(-1,)), (25, 50, 99), interpolation='midpoint')
# Observe the matrix and find that the larger ones are very large and the smaller ones are more concentrated. Add 0.001 to the resulting matrix, then do the log process, then do the min_max scaling process, and finally change the value of 0.01 back to 0
min_max_scaler = preprocessing.MinMaxScaler()
map_values = np.log(np.array(list(np.array(disease_food_infer).reshape(-1,)))+0.001)
inference_score = np.array(min_max_scaler.fit_transform(map_values.reshape([len(map_values),1]))).reshape([1,len(map_values)])
scaler_infer_com = 0.01 + np.array(list(inference_score[0]))*0.99 

<Figure size 432x288 with 1 Axes>

In [45]:
print("原来四分位点和99分位点：",np.percentile(list(np.array(disease_food_infer).reshape(-1,)), (25, 50, 75,99), interpolation='midpoint'))
print("进行log处理后的四分位点：", np.percentile(map_values, (25, 50, 75,99), interpolation='midpoint'))
print("进行缩放后对四分位点和99分位点",np.percentile(scaler_infer_com, (25, 50, 75,99), interpolation='midpoint'))
print("最终的最小值",scaler_infer_com.min())

原来四分位点和99分位点： [ 0.07489121  0.45278261  1.7069667  49.11030997]
进行log处理后的四分位点： [-2.57845435 -0.79013703  0.5353036   3.89408935]
进行缩放后对四分位点和99分位点 [0.30188784 0.42245883 0.51182199 0.73827627]
最终的最小值 0.01


In [47]:
disease_food_infer.shape

(684, 5840)

In [49]:
# disease_food_infer.to_csv('disease_food_infer_4.csv')